(C:ssp-traitement)=
# Traitement des signaux bruités



Détection de signaux 

Détecté un signal égal est-il présent 

Détection de signaux périodique inconnu de période inconnue 

L'objectif est de déterminer si un signal observé YN correspond à un signal XN périodique ébruiter 

On ne connaît pas la période de X 

La détection est obtenue par autocorrélation 

Si x est présent alors 

On rappelle que là corrélation revient à mesurer la ressemblance de 2 signaux dont l'un est décalé de M échantillons point donc : 

Air XX égale autocorrélation de X périodique de même période que x ceux de ligne air x br BX égale Inter corrélation entre x et B nul car les signaux sont décorrélés ce point 

Air bébé autocorrélation du bruit nul sauf on aime égal 0 car échantillons indépendants donc décorer les 

Détection de signaux périodiques inconnu de période connue 

Ce cas est plus favorable que le précédent car on dispose d'une information supplémentaire 

On cherche à déterminer si le signal observé YN correspond au signal périodique XL (de période t ) brut et par un bruit blanc BN : sauts de lignes le principe de détection consiste à effectuer l intercorrélation du y avec un signal auxiliaire zn périodique et de période t 

Détection d'un motif 

Un motif est un signal déterministe connu et de durée fini saute ligne le signal observé y est modélisé comme la somme du motif x placé en carré d'un bruit B 

On peut déterminer l'instant cas où le motif apparaît en calculant l'Inter corrélation entre y et XCF cours 1A convolution par XN moins en moins n 

Cette technique est appelée filtrage adapté (matched filter) elle est la solution optimale pour maximiser le RSB dans le cadre d'un bruit additif. Application radar Télécom 

Les bruitages de signaux 

Débrutées différents détectés 

Des bruitages d'un signal unique 

On n'observe : 

Pour réduire le bruit dans le signal (réduire la puissance donc là variance du bruit) on peut remplacer chaque échantillon YN par la moyenne de l échantillon autour de n : 

Ou n un perd 

On peut montrer que cette technique peut s'écrire sous la forme d'une convolution : c'est donc un filtrage appelé filtre moyenne EUR point la réponse impulsionnelle du filtre et une porte de longueur M point là TF t'aider de cette réponse impulsionnelle et ainsi Nice cardinale filtre passe bas 

Les bruitages de plusieurs réalisation d'un même signal ceux de ligne dans le cas où on dispose de I mesure d'un même signal x : 

Et que le bruit et s'en est centré () et de variance Sigma bakaré () alors la moyenne des y qui permet d'augmenter le RSB 

La puissance du bruit est divisée par qui 

Approximation par moindres carrés 

Gosse 1795 

On dispose de données y brut est tel que 

Ou F est une fonction paramétrique (sinusoïde polynôme) connu 

FN est la valeur de F 1N 

Titre est un ensemble de paramètres inconnu tête à sous de ligne le bruit bébé et de moyenne nulle 

Exemple 

Polynôme de degré 2 sauts de ligne exponentielle 

L'objectif est de retrouver ta point il est logique de choseaisir téta comme le paramètre qui minimise la somme des différences au carré 

La détermination d'état lorsque effet linéaire en état et simple exemple parabole point mais c'est beaucoup plus compliqué lorsque F n'est pas linéaire (coefficient éteint dans le exponentielle) dans le cas où la fonction F est linéaire son état il existe une expression explicite de la solution FTK égal 

Il faut faire les calculs avec la représentation vectorielle 

 